In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn import metrics 

In [2]:
rfc_data = pd.read_csv('~/data/downtownrecovery/curated_data/model_data_20230705.csv')
lq_clusters =  pd.read_csv('~/data/downtownrecovery/recovery_clusters/lq_clusters_single_period_1015.csv')
rq_dwtn_clusters = pd.read_csv('~/data/downtownrecovery/recovery_clusters/rq_dwtn_clusters_0822.csv')
rq_city_clusters =  pd.read_csv('~/data/downtownrecovery/recovery_clusters/rq_city_clusters_0822.csv')
rq_dwtn_clusters_period_1 = pd.read_csv('~/data/downtownrecovery/recovery_clusters/rq_dwtn_clusters_1015_period_1.csv')
rq_dwtn_clusters_period_2 =  pd.read_csv('~/data/downtownrecovery/recovery_clusters/rq_dwtn_clusters_1015_period_2_.csv')

In [3]:
rfc_data.columns

Index(['Unnamed: 0.1', 'Column1', 'Unnamed: 0', 'city',
       'normalized_visits_by_total_visits', 'week', 'metric', 'region',
       'metro_size', 'display_title', 'total_pop_downtown', 'total_pop_city',
       'pct_singlefam_downtown', 'pct_singlefam_city', 'pct_multifam_downtown',
       'pct_multifam_city', 'pct_mobile_home_and_others_downtown',
       'pct_mobile_home_and_others_city', 'pct_renter_downtown',
       'pct_renter_city', 'median_age_downtown', 'median_age_city',
       'bachelor_plus_downtown', 'bachelor_plus_city', 'median_hhinc_downtown',
       'median_hhinc_city', 'median_rent_downtown', 'median_rent_city',
       'pct_vacant_downtown', 'pct_vacant_city', 'pct_nhwhite_downtown',
       'pct_nhwhite_city', 'pct_nhblack_downtown', 'pct_nhblack_city',
       'pct_nhasian_downtown', 'pct_nhasian_city', 'pct_hisp_downtown',
       'pct_hisp_city', 'pct_commute_auto_downtown', 'pct_commute_auto_city',
       'pct_commute_public_transit_downtown',
       'pct_commute_pu

In [4]:
# convert / km2 back to m2 for the sake of model consistency
rfc_data['population_density_downtown'] = rfc_data['population_density_downtown'] / 1000000
rfc_data['population_density_city'] = rfc_data['population_density_city'] / 1000000
rfc_data['housing_density_downtown'] = rfc_data['housing_density_downtown'] / 1000000
rfc_data['housing_density_city'] = rfc_data['housing_density_city'] / 1000000
rfc_data['employment_density_downtown'] = rfc_data['employment_density_downtown'] / 1000000


In [5]:
rfc_data = rfc_data.drop(columns = ['Unnamed: 0.1', 'Column1', 'Unnamed: 0'])

In [6]:
rfc_data['display_name'] = rfc_data['city'].map({'Boston': "Boston, MA",
 'Portland': "Portland, OR",
 'Las Vegas': "Las Vegas, NV",
 'Detroit': "Detroit, MI",
 'New York': "New York, NY",
 'Los Angeles': "Los Angeles, CA",
 'Chicago': "Chicago, IL",
 'Houston': "Houston, TX",
 'Phoenix': "Phoenix, AZ",
 'Philadelphia': "Philadelphia, PA",
 'Vancouver': "Vancouver, BC",
 'Montreal': "Montreal, QC",
 'Calgary': "Calgary, AB",
 'Halifax': "Halifax, NS",
 'London': "London, ON",
 'Edmonton': "Edmonton, AB",
 'Mississauga': "Missisauga, ON",
 'Ottawa': "Ottawa, ON",
 'Winnipeg': "Winnipeg, MB",
 'Toronto': "Toronto, ON",
 'Quebec': "Quebec, QC",
 'Cleveland': "Cleveland, OH",
 'Honolulu': "Honolulu, HI",
 'Cincinnati': "Cincinnati, OH",
 'Pittsburgh': "Pittsburgh, PA",
 'Salt Lake City': "Salt Lake City, UT",
 'Fort Worth': "Forth Worth, TX",
 'Columbus': "Columbus, OH",
 'Indianapolis': "Indianapolis, IN",
 'Charlotte': "Charlotte, NC",
 'San Francisco': "San Francisco, CA",
 'Seattle': "Seattle, WA",
 'Denver': "Denver, CO",
 'Washington DC': "Washington DC",
 'Sacramento': "Sacramento, CA",
 'Kansas City': "Kansas City, MO",
 'Atlanta': "Atlanta, GA",
 'Omaha': "Omaha, NB",
 'Colorado Springs': "Colorado Springs, CO",
 'Raleigh': "Raleigh, NC",
 'Miami': "Miami, FL",
 'Memphis': "Memphis, TN",
 'St Louis': "St Louis, MO",
 'Orlando': "Orlando, FL",
 'San Antonio': "San Antonio, TX",
 'San Diego': "San Diego, CA",
 'Dallas': "Dallas, TX",
 'San Jose': "San Jose, CA",
 'Austin': "Austin, TX",
 'Jacksonville': "Jacksonville, FL",
 'Tulsa': "Tulsa, OK",
 'Bakersfield': "Bakersfield, CA",
 'Wichita': "Wichita, KS",
 'Tampa': "Tampa, FL",
 'New Orleans': "New Orleans, LA",
 'Nashville': "Nashville, TN",
 'Oklahoma City': "Oklahoma City, OK",
 'El Paso': "El Paso, TX",
 'Louisville': "Louisville, KY",
 'Baltimore': "Baltimore, MD",
 'Milwaukee': "Milwaukee, WI",
 'Albuquerque': "Albuquerque, NM",
 'Tucson': "Tucson, AZ",
 'Fresno': "Fresno, CA",
 'Oakland': "Oakland, CA",
 'Minneapolis':"Minneapolis, MN"})

In [7]:
display(rfc_data)

,city,normalized_visits_by_total_visits,week,metric,region,metro_size,display_title,total_pop_downtown,total_pop_city,pct_singlefam_downtown,...,summer_avg_temp,fall_avg_temp,normalized Violent crimes based on population,"Violent crimes per 100,000 people",population_density_downtown,population_density_city,housing_density_downtown,housing_density_city,employment_density_downtown,display_name
0,Albuquerque,0.905150,4/10/2023,downtown,Southwest,medium,"Albuquerque, NM",19191,720072,48.768244,...,78.066667,60.900000,0.005862,586.1826,0.001139,0.000412,0.000639,0.000177,0.005327,"Albuquerque, NM"
1,Atlanta,0.431086,4/10/2023,downtown,Southeast,medium,"Atlanta, GA",6628,1059958,2.933673,...,82.666667,68.016667,0.004039,403.9459,0.002452,0.001084,0.000580,0.000517,0.054370,"Atlanta, GA"
2,Austin,0.423679,4/10/2023,downtown,Southwest,large,"Austin, TX",10848,1586610,2.064277,...,88.800000,73.883333,0.001951,195.1006,0.002572,0.000539,0.001918,0.000216,0.044857,"Austin, TX"
3,Bakersfield,1.248455,4/10/2023,downtown,Pacific,medium,"Bakersfield, CA",12325,607203,42.991357,...,83.883333,69.466667,0.002235,223.5425,0.001138,0.000182,0.000491,0.000059,0.005146,"Bakersfield, CA"
4,Baltimore,0.855382,4/10/2023,downtown,Northeast,medium,"Baltimore, MD",39040,1022146,26.275475,...,77.733333,61.450000,0.008675,867.4740,0.005239,0.002061,0.002943,0.000938,0.018630,"Baltimore, MD"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58,Tulsa,0.674702,4/10/2023,downtown,Southwest,medium,"Tulsa, OK",5264,670092,15.473441,...,84.583333,67.350000,0.004944,494.3606,0.001498,0.000288,0.000986,0.000126,0.011028,"Tulsa, OK"
59,Vancouver,0.475278,4/10/2023,downtown,Canada,medium,"Vancouver, BC",126465,659685,2.452881,...,64.316667,52.933333,0.002900,289.9804,0.009483,0.003342,0.005609,0.001497,0.000920,"Vancouver, BC"
60,Washington DC,0.611956,4/10/2023,downtown,Northeast,large,Washington DC,20576,1159965,4.522350,...,79.600000,63.400000,0.005814,581.4000,0.007231,0.002946,0.004702,0.001302,0.115824,Washington DC
61,Wichita,0.754039,4/10/2023,downtown,Midwest,medium,"Wichita, KS",1751,481827,2.069950,...,84.466667,65.566667,0.005195,519.4958,0.000684,0.000325,0.000547,0.000139,0.018912,"Wichita, KS"


In [8]:
downtown_rfc_table = rfc_data[['city','total_pop_downtown', 'pct_singlefam_downtown', 'pct_multifam_downtown','pct_renter_downtown','median_age_downtown','bachelor_plus_downtown','median_hhinc_downtown','median_rent_downtown','pct_vacant_downtown','median_year_structure_built','median_no_rooms','pct_commute_auto_city','pct_commute_public_transit_city','pct_commute_bicycle_city','pct_commute_walk_city','housing_units_downtown','average_commute_time_city','pct_jobs_agriculture_forestry_fishing_hunting','pct_jobs_mining_quarrying_oil_gas', 'pct_jobs_utilities','pct_jobs_construction', 'pct_jobs_manufacturing','pct_jobs_wholesale_trade', 'pct_jobs_retail_trade','pct_jobs_transport_warehouse', 'pct_jobs_information','pct_jobs_finance_insurance','pct_jobs_real_estate','pct_jobs_professional_science_techical','pct_jobs_management_of_companies_enterprises','pct_jobs_administrative_support_waste','pct_jobs_educational_services','pct_jobs_healthcare_social_assistance','pct_jobs_arts_entertainment_recreation','pct_jobs_accomodation_food_services','pct_jobs_public_administration', 'employment_entropy', 'population_density_downtown', 'employment_density_downtown', 'housing_density_downtown','days_school_closing', 'days_workplace_closing','days_cancel_large_events', 'days_cancel_all_events','days_stay_home_requirements', 'days_income_support','days_mask_mandates', 'winter_avg_temp', 'summer_avg_temp']]

#'pct_nhwhite_downtown','pct_nhblack_downtown','pct_nhasian_downtown','pct_hisp_downtown',

city_rfc_table = rfc_data[['city','total_pop_city', 'pct_singlefam_city', 'pct_multifam_city','pct_renter_city','median_age_city','bachelor_plus_city','median_hhinc_city','median_rent_city','pct_vacant_city','pct_commute_auto_city','pct_commute_public_transit_city','pct_commute_bicycle_city','pct_commute_walk_city','housing_units_city','average_commute_time_city','pct_jobs_agriculture_forestry_fishing_hunting','pct_jobs_mining_quarrying_oil_gas', 'pct_jobs_utilities','pct_jobs_construction', 'pct_jobs_manufacturing','pct_jobs_wholesale_trade', 'pct_jobs_retail_trade','pct_jobs_transport_warehouse', 'pct_jobs_information','pct_jobs_finance_insurance','pct_jobs_real_estate','pct_jobs_professional_science_techical','pct_jobs_management_of_companies_enterprises','pct_jobs_administrative_support_waste','pct_jobs_educational_services','pct_jobs_healthcare_social_assistance','pct_jobs_arts_entertainment_recreation','pct_jobs_accomodation_food_services','pct_jobs_public_administration', 'employment_entropy', 'population_density_city', 'employment_density_downtown', 'housing_density_city', 'days_school_closing', 'days_workplace_closing','days_cancel_large_events', 'days_cancel_all_events','days_stay_home_requirements', 'days_income_support','days_mask_mandates', 'winter_avg_temp', 'summer_avg_temp']]

#'pct_nhwhite_city','pct_nhblack_city','pct_nhasian_city','pct_hisp_city',

lq_rfc_table = rfc_data[['city','total_pop_downtown', 'pct_singlefam_downtown', 'pct_multifam_downtown','pct_renter_downtown','median_age_downtown','bachelor_plus_downtown','median_hhinc_downtown','median_rent_downtown','pct_vacant_downtown','median_year_structure_built','median_no_rooms','pct_commute_auto_city','pct_commute_public_transit_city','pct_commute_bicycle_city','pct_commute_walk_city','housing_units_downtown','average_commute_time_city','pct_jobs_agriculture_forestry_fishing_hunting','pct_jobs_mining_quarrying_oil_gas', 'pct_jobs_utilities','pct_jobs_construction', 'pct_jobs_manufacturing','pct_jobs_wholesale_trade', 'pct_jobs_retail_trade','pct_jobs_transport_warehouse', 'pct_jobs_information','pct_jobs_finance_insurance','pct_jobs_real_estate','pct_jobs_professional_science_techical','pct_jobs_management_of_companies_enterprises','pct_jobs_administrative_support_waste','pct_jobs_educational_services','pct_jobs_healthcare_social_assistance','pct_jobs_arts_entertainment_recreation','pct_jobs_accomodation_food_services','pct_jobs_public_administration', 'employment_entropy', 'population_density_downtown', 'employment_density_downtown', 'housing_density_downtown','days_school_closing', 'days_workplace_closing','days_cancel_large_events', 'days_cancel_all_events','days_stay_home_requirements', 'days_income_support','days_mask_mandates', 'winter_avg_temp', 'summer_avg_temp']]

#lq_rfc_table = rfc_data[['city', 'total_pop_downtown', 'total_pop_city','pct_singlefam_downtown', 'pct_singlefam_city', 'pct_multifam_downtown','pct_multifam_city','pct_mobile_home_and_others_city', 'pct_renter_downtown','pct_renter_city', 'median_age_downtown', 'median_age_city','bachelor_plus_downtown', 'bachelor_plus_city', 'median_hhinc_downtown','median_hhinc_city', 'median_rent_downtown', 'median_rent_city','pct_vacant_downtown', 'pct_vacant_city', 'pct_commute_auto_downtown', 'pct_commute_auto_city','pct_commute_public_transit_downtown','pct_commute_public_transit_city', 'pct_commute_bicycle_downtown','pct_commute_bicycle_city', 'pct_commute_walk_downtown','pct_commute_walk_city','pct_commute_others_city', 'housing_units_downtown','housing_units_city', 'average_commute_time_downtown','average_commute_time_city','pct_jobs_agriculture_forestry_fishing_hunting','pct_jobs_mining_quarrying_oil_gas', 'pct_jobs_utilities','pct_jobs_construction', 'pct_jobs_manufacturing','pct_jobs_wholesale_trade', 'pct_jobs_retail_trade','pct_jobs_transport_warehouse', 'pct_jobs_information','pct_jobs_finance_insurance', 'pct_jobs_real_estate','pct_jobs_professional_science_techical','pct_jobs_management_of_companies_enterprises','pct_jobs_administrative_support_waste','pct_jobs_educational_services','pct_jobs_healthcare_social_assistance','pct_jobs_arts_entertainment_recreation','pct_jobs_accomodation_food_services','pct_jobs_public_administration', 'employment_entropy','population_density_downtown', 'population_density_city','employment_density_downtown', 'housing_density_downtown','housing_density_city', 'days_school_closing', 'days_workplace_closing','days_cancel_large_events', 'days_cancel_all_events','days_stay_home_requirements', 'days_income_support','days_mask_mandates', 'winter_avg_temp', 'summer_avg_temp']]

#'pct_nhwhite_downtown','pct_nhwhite_city', 'pct_nhblack_downtown', 'pct_nhblack_city','pct_nhasian_downtown', 'pct_nhasian_city', 'pct_hisp_downtown','pct_hisp_city', 

#lq_rfc_table['population_lq'] = lq_rfc_table['total_pop_downtown']/lq_rfc_table['total_pop_city']

In [9]:
rq_dwtn_clusters = rq_dwtn_clusters.replace('Québec','Quebec')
rq_dwtn_clusters = rq_dwtn_clusters.replace('Montréal','Montreal')
downtown_rfc_table = downtown_rfc_table[~downtown_rfc_table['city'].isin(['Dallas','Orlando','Mississauga','Hamilton','Oklahoma City'])]
downtown_rfc_table.loc[:,'cluster'] = downtown_rfc_table.loc[:,'city'].map(dict(zip(rq_dwtn_clusters['city'], rq_dwtn_clusters['cluster'])))

In [10]:
rq_city_clusters = rq_city_clusters.replace('Québec','Quebec')
rq_city_clusters = rq_city_clusters.replace('Montréal','Montreal')
city_rfc_table = city_rfc_table[~city_rfc_table['city'].isin(['Dallas','Orlando','Mississauga','Hamilton','Oklahoma City'])]
city_rfc_table.loc[:,'cluster'] = city_rfc_table.loc[:,'city'].map(dict(zip(rq_city_clusters['city'], rq_city_clusters['cluster'])))

In [11]:
lq_clusters = lq_clusters.replace('Québec','Quebec')
lq_clusters = lq_clusters.replace('Montréal','Montreal')
lq_clusters['cluster_map'] = lq_clusters['cluster'].map({0:"L1",1:"L3",2:"L2",3:"L4",4:"L1",5:"L1"})
lq_rfc_table = lq_rfc_table[~lq_rfc_table['city'].isin(['Dallas','Orlando','Mississauga','Hamilton','Oklahoma City'])].copy()
lq_rfc_table.loc[:,'cluster'] = lq_rfc_table.loc[:,'city'].map(dict(zip(lq_clusters['city'], lq_clusters['cluster_map'])))

In [12]:
rq_dwtn_clusters_period_1 = rq_dwtn_clusters_period_1.replace('Québec','Quebec')
rq_dwtn_clusters_period_1 = rq_dwtn_clusters_period_1.replace('Montréal','Montreal')
rq_dwtn_clusters_period_1['cluster_map'] = rq_dwtn_clusters_period_1['cluster'].map({0:"R1.1",1:"R1.5",2:"R1.3",3:"R1.4",4:"R1.2",5:"R1.1"})
downtown_rfc_table = downtown_rfc_table[~downtown_rfc_table['city'].isin(['Dallas','Orlando','Mississauga','Hamilton','Oklahoma City'])]
downtown_rfc_table.loc[:,'cluster_period_1'] = downtown_rfc_table.loc[:,'city'].map(dict(zip(rq_dwtn_clusters_period_1['city'], rq_dwtn_clusters_period_1['cluster_map'])))

In [13]:
rq_dwtn_clusters_period_2 = rq_dwtn_clusters_period_2.replace('Québec','Quebec')
rq_dwtn_clusters_period_2 = rq_dwtn_clusters_period_2.replace('Montréal','Montreal')
rq_dwtn_clusters_period_2['cluster_map'] = rq_dwtn_clusters_period_2['cluster'].map({0:"R2.5",1:"R2.1",2:"R2.3",3:"R2.1",4:"R2.4",5:"R2.2"})
downtown_rfc_table = downtown_rfc_table[~downtown_rfc_table['city'].isin(['Dallas','Orlando','Mississauga','Hamilton','Oklahoma City'])]
downtown_rfc_table.loc[:,'cluster_period_2'] = downtown_rfc_table.loc[:,'city'].map(dict(zip(rq_dwtn_clusters_period_2['city'], rq_dwtn_clusters_period_2['cluster_map'])))

In [79]:
set_seed = 20230705

In [70]:

from sklearn.model_selection import RandomizedSearchCV
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]

# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

X = downtown_rfc_table.drop(columns=['city','cluster','cluster_period_1','cluster_period_2']).to_numpy()
y = downtown_rfc_table['cluster_period_2'].to_numpy()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = set_seed)

rf = RandomForestClassifier( random_state = set_seed)
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2,  random_state = set_seed, n_jobs = -1)
rf_random.fit(X_train, y_train)
rf_random.best_params_

Fitting 3 folds for each of 100 candidates, totalling 300 fits


{'n_estimators': 1000,
 'min_samples_split': 2,
 'min_samples_leaf': 1,
 'max_features': 'sqrt',
 'max_depth': 40,
 'bootstrap': False}

In [80]:
def generate_random_forest_classifier(X, y, indextable):
    sc = StandardScaler()
    X = sc.fit_transform(X)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.001, 
                                                        random_state = set_seed
                                                        )
    model = RandomForestClassifier(n_estimators = 1600,
                                   min_samples_split = 2,
                                   min_samples_leaf = 4,
                                   max_features = 'sqrt',
                                   random_state = set_seed,
                                   max_depth = 30,
                                   bootstrap = True) 
    model.fit(X_train, y_train)
    train_score = model.score(X_train, y_train)
    test_score = model.score(X_test, y_test)
    #feature_importances = pd.Series(model.feature_importances_, index = indextable.drop(columns=['city','cluster','cluster_period_1','cluster_period_2']).columns).sort_values(ascending = False)
    feature_importances = pd.Series(model.feature_importances_, index = indextable.columns).sort_values(ascending = False) #,'cluster_period_1','cluster_period_2'
    return feature_importances, train_score, test_score

In [81]:
#Generate RFC Downtown
X_dwtn = downtown_rfc_table.drop(columns=['city','cluster','cluster_period_1','cluster_period_2']).to_numpy()
y_dwtn = downtown_rfc_table['cluster'].to_numpy()
feature_importances, train_score, test_score = generate_random_forest_classifier(X_dwtn,y_dwtn, downtown_rfc_table.drop(columns=['city','cluster','cluster_period_1','cluster_period_2']))
print('train score: '+str(train_score))
print('test score: '+str(test_score))
feature_importances[0:60]

train score: 0.9508196721311475
test score: 0.0


pct_jobs_professional_science_techical           0.057572
pct_jobs_information                             0.050121
pct_jobs_construction                            0.035680
pct_jobs_public_administration                   0.033150
pct_commute_public_transit_city                  0.032144
pct_multifam_downtown                            0.031001
pct_singlefam_downtown                           0.030316
pct_jobs_finance_insurance                       0.028916
pct_commute_auto_city                            0.028003
pct_jobs_agriculture_forestry_fishing_hunting    0.026916
pct_commute_walk_city                            0.025612
pct_vacant_downtown                              0.025106
pct_jobs_accomodation_food_services              0.024689
pct_commute_bicycle_city                         0.023447
winter_avg_temp                                  0.023155
employment_density_downtown                      0.022363
pct_jobs_real_estate                             0.022251
pct_renter_dow

In [82]:
#Generate RFC City
X_city = city_rfc_table.drop(columns=['city','cluster']).to_numpy()
y_city = city_rfc_table['cluster'].to_numpy()
feature_importances, train_score, test_score = generate_random_forest_classifier(X_city,y_city, city_rfc_table.drop(columns=['city','cluster']))
print('train score: '+str(train_score))
feature_importances[0:60]

train score: 0.9016393442622951


pct_jobs_information                             0.050442
employment_density_downtown                      0.045189
pct_jobs_retail_trade                            0.038892
bachelor_plus_city                               0.038486
pct_commute_bicycle_city                         0.033938
days_cancel_large_events                         0.031749
housing_density_city                             0.030352
population_density_city                          0.030111
days_stay_home_requirements                      0.025609
pct_commute_public_transit_city                  0.025482
pct_singlefam_city                               0.025380
pct_jobs_educational_services                    0.024890
pct_jobs_public_administration                   0.024854
pct_multifam_city                                0.024525
pct_jobs_finance_insurance                       0.023867
pct_commute_auto_city                            0.023572
pct_renter_city                                  0.023361
days_workplace

In [86]:
#Generate RFC LQ
X_lq = lq_rfc_table.drop(columns=['city','cluster']).to_numpy()
y_lq = lq_rfc_table['cluster'].to_numpy()
feature_importances, train_score, test_score = generate_random_forest_classifier(X_lq,y_lq, lq_rfc_table.drop(columns=['city','cluster']))
print('train score: '+str(train_score))
feature_importances[0:60]

train score: 0.9672131147540983


pct_jobs_professional_science_techical           0.058055
median_hhinc_downtown                            0.054598
winter_avg_temp                                  0.045629
bachelor_plus_downtown                           0.044698
average_commute_time_city                        0.043421
median_rent_downtown                             0.038961
employment_density_downtown                      0.036090
pct_jobs_information                             0.034514
pct_jobs_educational_services                    0.031871
pct_jobs_manufacturing                           0.030593
pct_jobs_public_administration                   0.026736
pct_multifam_downtown                            0.022830
pct_jobs_accomodation_food_services              0.022757
pct_jobs_retail_trade                            0.021746
housing_units_downtown                           0.020838
pct_jobs_finance_insurance                       0.020013
pct_jobs_utilities                               0.018879
pct_singlefam_

In [84]:
#Generate RFC Downtown Period 1 Jun 20 - May 21
X_dwtn = downtown_rfc_table.drop(columns=['city','cluster','cluster_period_1','cluster_period_2','pct_singlefam_downtown']).to_numpy()
y_dwtn = downtown_rfc_table['cluster_period_1'].to_numpy()
feature_importances, train_score, test_score = generate_random_forest_classifier(X_dwtn,y_dwtn, downtown_rfc_table.drop(columns=['city','cluster','cluster_period_1','cluster_period_2','pct_singlefam_downtown']))
print('train score: '+str(train_score))
print('test score: '+str(test_score))
feature_importances.to_csv('~/data/downtownrecovery/rfc_outputs/downtown_period1.csv')
display(pd.DataFrame(feature_importances.reset_index()))

train score: 0.9508196721311475
test score: 0.0


,index,0
0,pct_jobs_professional_science_techical,0.059180
1,pct_jobs_information,0.052550
2,pct_multifam_downtown,0.043947
3,pct_commute_public_transit_city,0.037759
4,pct_commute_auto_city,0.033830
5,pct_jobs_educational_services,0.033622
6,pct_jobs_construction,0.033447
7,pct_renter_downtown,0.032330
8,pct_jobs_public_administration,0.030122
9,pct_commute_walk_city,0.029627


In [85]:
#Generate RFC Downtown Period 2 Jun 21 - May 22
X_dwtn = downtown_rfc_table.drop(columns=['city','cluster','cluster_period_1','cluster_period_2','pct_singlefam_downtown']).to_numpy()
y_dwtn = downtown_rfc_table['cluster_period_2'].to_numpy()
feature_importances, train_score, test_score = generate_random_forest_classifier(X_dwtn,y_dwtn, downtown_rfc_table.drop(columns=['city','cluster','cluster_period_1','cluster_period_2','pct_singlefam_downtown']))
print('train score: '+str(train_score))
print('test score: '+str(test_score))
feature_importances.to_csv('~/data/downtownrecovery/rfc_outputs/downtown_period2.csv')
display(pd.DataFrame(feature_importances.reset_index()))

train score: 0.9344262295081968
test score: 0.0


,index,0
0,pct_jobs_professional_science_techical,0.065377
1,pct_commute_auto_city,0.039129
2,pct_multifam_downtown,0.038729
3,summer_avg_temp,0.038043
4,employment_density_downtown,0.036255
5,pct_commute_public_transit_city,0.035166
6,pct_commute_bicycle_city,0.032596
7,pct_jobs_public_administration,0.030604
8,pct_commute_walk_city,0.029878
9,days_income_support,0.028252


In [87]:
#Generate RFC LQ
X_dwtn = lq_rfc_table.drop(columns=['city','cluster','pct_singlefam_downtown']).to_numpy()
y_dwtn = lq_rfc_table['cluster'].to_numpy()
feature_importances, train_score, test_score = generate_random_forest_classifier(X_dwtn,y_dwtn, lq_rfc_table.drop(columns=['city','cluster','pct_singlefam_downtown']))
print('train score: '+str(train_score))
print('test score: '+str(test_score))
feature_importances.to_csv('~/data/downtownrecovery/rfc_outputs/lq.csv')
display(pd.DataFrame(feature_importances.reset_index()))

train score: 0.9672131147540983
test score: 0.0


,index,0
0,pct_jobs_professional_science_techical,0.053966
1,median_hhinc_downtown,0.052499
2,winter_avg_temp,0.047190
3,bachelor_plus_downtown,0.043307
4,average_commute_time_city,0.042442
5,median_rent_downtown,0.036110
6,employment_density_downtown,0.036042
7,pct_jobs_information,0.035106
8,pct_jobs_educational_services,0.033044
9,pct_jobs_manufacturing,0.032216


In [ ]:
normalized_visits = get_table_as_pandas_df('0714_combined_metrics_df')

In [ ]:
normalized_visits_chronicle = normalized_visits[normalized_visits['metric']=='downtown'][['city','normalized_visits_by_total_visits','week']]
normalized_visits_chronicle = normalized_visits_chronicle[normalized_visits_chronicle['normalized_visits_by_total_visits'].notna()]
normalized_visits_chronicle['city'] = normalized_visits_chronicle['city'].replace('Québec','Quebec')
normalized_visits_chronicle = normalized_visits_chronicle[normalized_visits_chronicle['city'].isin(['Calgary','Quebec','San Francisco','Los Angeles','San Jose','Seattle','New York','Washington DC','Salt Lake City','Atlanta','Cleveland'])]
display(normalized_visits_chronicle.sort_values(['city','week']).reset_index(drop='True'))

In [ ]:
normalized_visits_chronicle['week'].value_counts()

In [ ]:
normalized_visits_by_state_scalingnormalized_visits[normalized_visits['metric']=='downtown']['normalized_visits_by_state_scaling'].isna().sum()

In [ ]:
normalized_visits = get_table_as_pandas_df('0714_combined_metrics_df')[['city','metric','normalized_visits_by_total_visits','week']].replace('Montréal','Montreal').replace('Québec','Quebec')
normalized_visits = normalized_visits[~normalized_visits["city"].isin(['Dallas','Orlando','Mississauga','Hamilton','Oklahoma City'])].copy()
normalized_visits_city = normalized_visits[normalized_visits["metric"]=="metro"].copy()
normalized_visits_city['cluster'] = normalized_visits_city['city'].map(dict(zip(rq_city_clusters['city'], rq_city_clusters['cluster']))).astype(int)

In [ ]:
city_rfc_table = city_rfc_table[~city_rfc_table['city'].isin(['Dallas','Orlando','Mississauga','Hamilton','Oklahoma City'])]
city_rfc_table.loc[:,'cluster'] = city_rfc_table.loc[:,'city'].map(dict(zip(rq_city_clusters['city'], rq_city_clusters['cluster'])))

In [ ]:
mean_city = normalized_visits_city.groupby(['cluster','week']).mean()['normalized_visits_by_total_visits'].reset_index()

In [ ]:
max_city = normalized_visits_city.groupby(['cluster','week']).max()['normalized_visits_by_total_visits'].reset_index()

In [ ]:
min_city = normalized_visits_city.groupby(['cluster','week']).min()['normalized_visits_by_total_visits'].reset_index()

In [ ]:
mean_city["week"] = pd.to_datetime(mean_city["week"])

In [ ]:
mean_city_rolling = pd.DataFrame()
for i in mean_city["cluster"].unique():
    mean_city_rolling_cluster = mean_city[mean_city["cluster"]==i].copy()
    mean_city_rolling_cluster['rolling'] = mean_city_rolling_cluster['normalized_visits_by_total_visits'].rolling(window=9).mean().shift(-4)
    mean_city_rolling = pd.concat([mean_city_rolling,mean_city_rolling_cluster])

In [ ]:
import seaborn as sns
#sns.set_style('darkgrid')
sns.set(rc={'figure.figsize':(14,8)})
ax = sns.lineplot(data=mean_city_rolling, x ='week', y = 'rolling', hue='cluster', palette='tab10')
plt.ylabel('City RQ')
plt.xlabel('Date')
plt.show()

In [ ]:
import datetime
import seaborn as sns
datetime.date(2020, 6, 1)

In [ ]:
normalized_visits_dwtn = normalized_visits[normalized_visits["metric"]=="downtown"].copy()
normalized_visits_dwtn['cluster'] = normalized_visits_dwtn['city'].map(dict(zip(rq_dwtn_clusters['city'], rq_dwtn_clusters['cluster']))).astype(int)
#mean_dwtn = normalized_visits_dwtn.groupby(['cluster','week']).mean()['normalized_visits_by_total_visits'].reset_index()
normalized_visits_dwtn["week"] = pd.to_datetime(normalized_visits_dwtn["week"]).dt.date
normalized_visits_dwtn[normalized_visits_dwtn.isnull()]

In [ ]:
 grouped.loc[:,('rolling', 'median')]

In [ ]:
normalized_visits_dwtn = normalized_visits[normalized_visits["metric"]=="downtown"].copy()
normalized_visits_dwtn['cluster'] = normalized_visits_dwtn['city'].map(dict(zip(rq_dwtn_clusters['city'], rq_dwtn_clusters['cluster']))).astype(int)
#mean_dwtn = normalized_visits_dwtn.groupby(['cluster','week']).mean()['normalized_visits_by_total_visits'].reset_index()
normalized_visits_dwtn["week"] = pd.to_datetime(normalized_visits_dwtn["week"]).dt.date
normalized_visits_dwtn = normalized_visits_dwtn.dropna()
normalized_visits_dwtn = normalized_visits_dwtn[normalized_visits_dwtn["week"]>datetime.date(2020, 6, 1)]
normalized_visits_dwtn['rolling'] = normalized_visits_dwtn.sort_values(by = 'week').groupby(['city'])['normalized_visits_by_total_visits'].transform(lambda x : x.rolling(window=19).mean().shift(-9))
normalized_visits_dwtn = normalized_visits_dwtn.dropna()

# compute the min, median and max
grouped = normalized_visits_dwtn.groupby(["cluster", "week"]).agg({'rolling': ['min', 'median', 'max']}).unstack("cluster")
grouped = grouped.dropna()

# plot medians
ax = grouped.loc[:,('rolling', 'median')].plot(figsize = (16, 12), sharey = True)

# Getting the color palette used
palette = sns.color_palette()

# index is now the number of clusters and indexes into grouped by cluster #
index = 0
for index in np.arange(0, len(normalized_visits_dwtn['cluster'].unique())):
        if index < 10: 
            ax.fill_between(grouped.index, grouped.loc[:,('rolling', 'median', index)], 
                    grouped.loc[:,('rolling', 'max', index)], alpha=.2, color=palette[index])
            ax.fill_between(grouped.index, 
                    grouped.loc[:,('rolling', 'min', index)] , grouped.loc[:,('rolling', 'median', index)], alpha=.2, color=palette[index])
plt.ylabel('Downtown RQ')
plt.xlabel('Date')
ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))
plt.title('Downtown recovery cluster extrema')
plt.show()

In [ ]:
normalized_visits_dwtn = normalized_visits[normalized_visits["metric"]=="downtown"].copy()
normalized_visits_dwtn['cluster'] = normalized_visits_dwtn['city'].map(dict(zip(rq_dwtn_clusters['city'], rq_dwtn_clusters['cluster']))).astype(int)
mean_dwtn = normalized_visits_dwtn.groupby(['cluster','week']).mean()['normalized_visits_by_total_visits'].reset_index()
mean_dwtn["week"] = pd.to_datetime(mean_dwtn["week"]).dt.date
mean_dwtn = mean_dwtn[mean_dwtn["week"]>datetime.date(2020, 6, 1)]
mean_dwtn['rolling'] = mean_dwtn.groupby('cluster')['normalized_visits_by_total_visits'].transform(lambda x : x.rolling(window=19).mean().shift(-9))
sns.set(rc={'figure.figsize':(14,10)})
ax = sns.lineplot(data=mean_dwtn, x ='week', y = 'rolling', hue='cluster', palette='tab10')
plt.ylabel('Downtown RQ')
plt.xlabel('Date')
plt.show()

In [ ]:
normalized_visits_dwtn[normalized_visits_dwtn["week"]==datetime.date(2020,12,7)]

In [ ]:
clusters = get_table_as_pandas_df('rq_dwtn_clusters_0823_period_2').replace('Montréal','Montreal').replace('Québec','Quebec')
normalized_visits_dwtn = normalized_visits[normalized_visits["metric"]=="downtown"].copy()
normalized_visits_dwtn['cluster'] = normalized_visits_dwtn['city'].map(dict(zip(clusters['city'], clusters['cluster']))).astype(int)
normalized_visits_dwtn['rolling'] = normalized_visits_dwtn.groupby('city')['normalized_visits_by_total_visits'].transform(lambda x : x.rolling(window=9).mean().shift(-4))
normalized_visits_dwtn["week"] = pd.to_datetime(normalized_visits_dwtn["week"]).dt.date
normalized_visits_dwtn.week.unique()

In [ ]:
clusters = get_table_as_pandas_df('rq_dwtn_clusters_0823_period_2').replace('Montréal','Montreal').replace('Québec','Quebec')
normalized_visits_dwtn = normalized_visits[normalized_visits["metric"]=="downtown"].copy()
normalized_visits_dwtn['cluster'] = normalized_visits_dwtn['city'].map(dict(zip(clusters['city'], clusters['cluster']))).astype(int)
normalized_visits_dwtn['rolling'] = normalized_visits_dwtn.groupby('city')['normalized_visits_by_total_visits'].transform(lambda x : x.rolling(window=5).mean().shift(-3))
normalized_visits_dwtn["week"] = pd.to_datetime(mean_dwtn["week"]).dt.date
#mean_dwtn = normalized_visits_dwtn.groupby(['cluster','week']).mean()['normalized_visits_by_total_visits'].reset_index()
#mean_dwtn["week"] = pd.to_datetime(mean_dwtn["week"]).dt.date
#mean_dwtn = mean_dwtn[mean_dwtn["week"]>datetime.date(2020, 6, 1)]
#mean_dwtn['rolling'] = mean_dwtn.groupby('cluster')['normalized_visits_by_total_visits'].transform(lambda x : x.rolling(window=9).mean().shift(-4))
sns.set(rc={'figure.figsize':(14,10)})
ax = sns.lineplot(data=normalized_visits_dwtn, x ='week', y = 'rolling', hue='cluster', palette='tab10')
plt.ylabel('Downtown RQ')
plt.xlabel('Date')
plt.show()

In [ ]:
clusters = get_table_as_pandas_df('rq_dwtn_clusters_0823_period_2').replace('Montréal','Montreal').replace('Québec','Quebec')
normalized_visits_dwtn = normalized_visits[normalized_visits["metric"]=="downtown"].copy()
normalized_visits_dwtn['cluster'] = normalized_visits_dwtn['city'].map(dict(zip(clusters['city'], clusters['cluster']))).astype(int)
normalized_visits_dwtn['rolling'] = normalized_visits_dwtn.groupby('city')['normalized_visits_by_total_visits'].transform(lambda x : x.rolling(window=5).mean().shift(-3))
#normalized_visits_dwtn["week"] = pd.to_datetime(mean_dwtn["week"]).dt.date
normalized_visits_dwtn["color"] = normalized_visits_dwtn["cluster"].map({0:'blue', 1:'green',2:'orange',3:'red',4:'pink',5:'black'})
sns.set(rc={'figure.figsize':(14,10)})
for city in normalized_visits_dwtn['city'].unique():
    city_data = normalized_visits_dwtn[normalized_visits_dwtn['city']==city]
    ax = plt.plot(city_data['week'], city_data['rolling'], color=city_data['color'].iloc[0], linewidth=0.5)
plt.ylabel('Downtown RQ')
plt.xlabel('Date')
plt.show()

In [ ]:
import seaborn as sns

In [ ]:
clusters = get_table_as_pandas_df('rq_dwtn_clusters_1015_period_1').replace('Montréal','Montreal').replace('Québec','Quebec')
normalized_visits = get_table_as_pandas_df('metrics_1015').replace('Montréal','Montreal').replace('Québec','Quebec')
normalized_visits = normalized_visits[~normalized_visits['city'].isin(['Orlando','Dallas','Hamilton','Mississauga','Oklahoma City'])]
normalized_visits_dwtn = normalized_visits[normalized_visits["metric"]=="downtown"].copy()
normalized_visits_dwtn['cluster'] = normalized_visits_dwtn['city'].map(dict(zip(clusters['city'], clusters['cluster']))).astype(int)
normalized_visits_dwtn['rolling'] = normalized_visits_dwtn.groupby('city')['normalized_visits_by_total_visits'].transform(lambda x : x.rolling(window=9).mean().shift(-4))
normalized_visits_dwtn = normalized_visits_dwtn.sort_values(by='week')
normalized_visits_dwtn = normalized_visits_dwtn[normalized_visits_dwtn['week']<'2022-06-01']
normalized_visits_dwtn["week"] = pd.to_datetime(normalized_visits_dwtn["week"].values)
sns.set(rc={'figure.figsize':(14,10)})
ax = sns.lineplot(data=normalized_visits_dwtn, x ='week', y = 'rolling', hue='cluster', palette='tab10', units='city', alpha=0.3, estimator=None)#, ci='sd')
plt.ylabel('Downtown RQ')
plt.xlabel('Date')
plt.show()

In [ ]:
clusters = get_table_as_pandas_df('rq_dwtn_clusters_0823_period_2').replace('Montréal','Montreal').replace('Québec','Quebec')
normalized_visits = get_table_as_pandas_df('metrics_1015').replace('Montréal','Montreal').replace('Québec','Quebec')
normalized_visits = normalized_visits[~normalized_visits['city'].isin(['Orlando','Dallas','Hamilton','Mississauga','Oklahoma City'])]
normalized_visits_dwtn = normalized_visits[normalized_visits["metric"]=="downtown"].copy()
normalized_visits_dwtn['cluster'] = normalized_visits_dwtn['city'].map(dict(zip(clusters['city'], clusters['cluster']))).astype(int)
normalized_visits_dwtn['rolling'] = normalized_visits_dwtn.groupby('city')['normalized_visits_by_total_visits'].transform(lambda x : x.rolling(window=9).mean().shift(-4))
normalized_visits_dwtn = normalized_visits_dwtn.sort_values(by='week')
normalized_visits_dwtn = normalized_visits_dwtn[normalized_visits_dwtn['week']<'2022-06-01']
normalized_visits_dwtn["week"] = pd.to_datetime(normalized_visits_dwtn["week"].values)
#mean_dwtn = normalized_visits_dwtn.groupby(['cluster','week']).mean()['normalized_visits_by_total_visits'].reset_index()
#mean_dwtn["week"] = pd.to_datetime(mean_dwtn["week"]).dt.date
#mean_dwtn = mean_dwtn[mean_dwtn["week"]>datetime.date(2020, 6, 1)]
#mean_dwtn['rolling'] = mean_dwtn.groupby('cluster')['normalized_visits_by_total_visits'].transform(lambda x : x.rolling(window=9).mean().shift(-4))
sns.set(rc={'figure.figsize':(14,10)})
ax = sns.lineplot(data=normalized_visits_dwtn, x ='week', y = 'rolling', hue='cluster', palette='tab10', units='city', alpha=0.3, estimator=None)#, ci='sd')
plt.ylabel('Downtown RQ')
plt.xlabel('Date')
plt.show()

In [ ]:
normalized_visits = get_table_as_pandas_df('0714_combined_metrics_df')[['city','metric','normalized_visits_by_total_visits','week']].replace('Montréal','Montreal').replace('Québec','Quebec')
normalized_visits = normalized_visits[~normalized_visits["city"].isin(['Dallas','Orlando','Mississauga','Hamilton','Oklahoma City'])].copy()
normalized_visits_lq = normalized_visits[normalized_visits["metric"]=="relative"].copy()
normalized_visits_lq['cluster'] = normalized_visits_city['city'].map(dict(zip(rq_city_clusters['city'], rq_city_clusters['cluster']))).astype(int)

In [ ]:
import seaborn as sns
sns.set_style("whitegrid")

In [ ]:
clusters = get_table_as_pandas_df('rq_dwtn_clusters_0823_period_1').replace('Montréal','Montreal').replace('Québec','Quebec')
normalized_visits = get_table_as_pandas_df('0714_combined_metrics_df').replace('Montréal','Montreal').replace('Québec','Quebec')
normalized_visits = normalized_visits[~normalized_visits['city'].isin(['Orlando','Dallas','Hamilton','Mississauga','Oklahoma City'])]
normalized_visits_dwtn = normalized_visits[normalized_visits["metric"]=="relative"].copy()
normalized_visits_dwtn['cluster'] = normalized_visits_dwtn['city'].map(dict(zip(clusters['city'], clusters['cluster']))).astype(int)
normalized_visits_dwtn['rolling'] = normalized_visits_dwtn.groupby('city')['normalized_visits_by_total_visits'].transform(lambda x : x.rolling(window=15).mean().shift(-7))
normalized_visits_dwtn = normalized_visits_dwtn.sort_values(by='week')
normalized_visits_dwtn = normalized_visits_dwtn[normalized_visits_dwtn['week']<'2022-06-01']
normalized_visits_dwtn["week"] = pd.to_datetime(normalized_visits_dwtn["week"].values)
#mean_dwtn = normalized_visits_dwtn.groupby(['cluster','week']).mean()['normalized_visits_by_total_visits'].reset_index()
#mean_dwtn["week"] = pd.to_datetime(mean_dwtn["week"]).dt.date
#mean_dwtn = mean_dwtn[mean_dwtn["week"]>datetime.date(2020, 6, 1)]
#mean_dwtn['rolling'] = mean_dwtn.groupby('cluster')['normalized_visits_by_total_visits'].transform(lambda x : x.rolling(window=9).mean().shift(-4))
sns.set(rc={'figure.figsize':(14,10)})
ax = sns.lineplot(data=normalized_visits_dwtn, x ='week', y = 'rolling', hue='cluster', palette='tab10', units='city', alpha=0.3, estimator=None)#, ci='sd')
plt.ylabel('LQ')
plt.xlabel('Date')
plt.show()

In [ ]:
clusters = get_table_as_pandas_df('rq_dwtn_clusters_0823_period_2').replace('Montréal','Montreal').replace('Québec','Quebec')
normalized_visits = get_table_as_pandas_df('0714_combined_metrics_df').replace('Montréal','Montreal').replace('Québec','Quebec')
normalized_visits = normalized_visits[~normalized_visits['city'].isin(['Orlando','Dallas','Hamilton','Mississauga','Oklahoma City'])]
normalized_visits_dwtn = normalized_visits[normalized_visits["metric"]=="relative"].copy()
normalized_visits_dwtn['cluster'] = normalized_visits_dwtn['city'].map(dict(zip(clusters['city'], clusters['cluster']))).astype(int)
normalized_visits_dwtn['rolling'] = normalized_visits_dwtn.groupby('city')['normalized_visits_by_total_visits'].transform(lambda x : x.rolling(window=15).mean().shift(-7))
normalized_visits_dwtn = normalized_visits_dwtn.sort_values(by='week')
normalized_visits_dwtn = normalized_visits_dwtn[normalized_visits_dwtn['week']<'2022-06-01']
normalized_visits_dwtn["week"] = pd.to_datetime(normalized_visits_dwtn["week"].values)
#mean_dwtn = normalized_visits_dwtn.groupby(['cluster','week']).mean()['normalized_visits_by_total_visits'].reset_index()
#mean_dwtn["week"] = pd.to_datetime(mean_dwtn["week"]).dt.date
#mean_dwtn = mean_dwtn[mean_dwtn["week"]>datetime.date(2020, 6, 1)]
#mean_dwtn['rolling'] = mean_dwtn.groupby('cluster')['normalized_visits_by_total_visits'].transform(lambda x : x.rolling(window=9).mean().shift(-4))
sns.set(rc={'figure.figsize':(14,10)})
ax = sns.lineplot(data=normalized_visits_dwtn, x ='week', y = 'rolling', hue='cluster', palette='tab10', units='city', alpha=0.3, estimator=None)#, ci='sd')
plt.ylabel('LQ')
plt.xlabel('Date')
plt.show()

In [ ]:
clusters = get_table_as_pandas_df('lq_clusters_0824_period_2_new').replace('Montréal','Montreal').replace('Québec','Quebec')
normalized_visits = get_table_as_pandas_df('0714_combined_metrics_df').replace('Montréal','Montreal').replace('Québec','Quebec')
normalized_visits = normalized_visits[~normalized_visits['city'].isin(['Orlando','Dallas','Hamilton','Mississauga','Oklahoma City'])]
normalized_visits_dwtn = normalized_visits[normalized_visits["metric"]=="relative"].copy()
normalized_visits_dwtn['cluster'] = normalized_visits_dwtn['city'].map(dict(zip(clusters['city'], clusters['cluster']))).astype(int)
normalized_visits_dwtn['rolling'] = normalized_visits_dwtn.groupby('city')['normalized_visits_by_total_visits'].transform(lambda x : x.rolling(window=15).mean().shift(-7))
normalized_visits_dwtn = normalized_visits_dwtn.sort_values(by='week')
normalized_visits_dwtn = normalized_visits_dwtn[normalized_visits_dwtn['week']<'2022-06-01']
normalized_visits_dwtn["week"] = pd.to_datetime(normalized_visits_dwtn["week"].values)
#mean_dwtn = normalized_visits_dwtn.groupby(['cluster','week']).mean()['normalized_visits_by_total_visits'].reset_index()
#mean_dwtn["week"] = pd.to_datetime(mean_dwtn["week"]).dt.date
#mean_dwtn = mean_dwtn[mean_dwtn["week"]>datetime.date(2020, 6, 1)]
#mean_dwtn['rolling'] = mean_dwtn.groupby('cluster')['normalized_visits_by_total_visits'].transform(lambda x : x.rolling(window=9).mean().shift(-4))
sns.set(rc={'figure.figsize':(14,10)})
ax = sns.lineplot(data=normalized_visits_dwtn, x ='week', y = 'rolling', hue='cluster', palette='tab10', units='city', alpha=0.3, estimator=None)#, ci='sd')
plt.ylabel('LQ')
plt.xlabel('Date')
plt.show()

In [ ]:
display(get_table_as_pandas_df('lq_clusters_single_period_4'))

In [ ]:
sns.set_theme(style="whitegrid")
clusters = get_table_as_pandas_df('lq_clusters_single_period_4').replace('Montréal','Montreal').replace('Québec','Quebec')
normalized_visits = get_table_as_pandas_df('0714_combined_metrics_df').replace('Montréal','Montreal').replace('Québec','Quebec')
normalized_visits = normalized_visits[~normalized_visits['city'].isin(['Orlando','Dallas','Hamilton','Mississauga','Oklahoma City'])]
normalized_visits_dwtn = normalized_visits[normalized_visits["metric"]=="relative"].copy()
normalized_visits_dwtn = normalized_visits_dwtn[normalized_visits_dwtn["normalized_visits_by_total_visits"].notna()]
normalized_visits_dwtn['cluster'] = normalized_visits_dwtn['city'].map(dict(zip(clusters['city'], clusters['cluster']))).astype(int)
normalized_visits_dwtn['rolling'] = normalized_visits_dwtn.groupby('city')['normalized_visits_by_total_visits'].transform(lambda x : x.rolling(window=13).mean().shift(-6))
normalized_visits_dwtn = normalized_visits_dwtn.sort_values(by='week')
#normalized_visits_dwtn = normalized_visits_dwtn[normalized_visits_dwtn['week']>'2020-11-01']
normalized_visits_dwtn = normalized_visits_dwtn[normalized_visits_dwtn['week']<'2022-06-01']
normalized_visits_dwtn["week"] = pd.to_datetime(normalized_visits_dwtn["week"])
normalized_visits_dwtn_avg = normalized_visits_dwtn.groupby(['week','cluster']).mean()['rolling'].reset_index()
sns.set(rc={'figure.figsize':(16,12)})
sns.set(font_scale = 1.15)
sns.set_style("whitegrid")
sns.lineplot(data=normalized_visits_dwtn, x ='week', y = 'rolling', hue='cluster', palette='tab10', units='city', alpha=0.25, estimator=None, linewidth=1.5)#, ci='sd')
sns.lineplot(data=normalized_visits_dwtn_avg, x ='week', y = 'rolling', hue='cluster', palette='tab10', units='cluster', alpha=0.8, estimator=None, linewidth=2.5)#, ci='sd')
plt.ylabel('Location Quotient')
plt.xlabel('Date (Year-Month)')
plt.legend(bbox_to_anchor=(1.02, 1), loc='upper left', borderaxespad=0)
plt.show()

In [ ]:
display(get_table_as_pandas_df('rq_dwtn_clusters_0823_period_1'))

In [ ]:
import seaborn as sns

In [ ]:
sns.set_theme(style="whitegrid")
clusters = get_table_as_pandas_df('rq_dwtn_clusters_1015_period_1').replace('Montréal','Montreal').replace('Québec','Quebec')
normalized_visits = get_table_as_pandas_df('metrics_1015').replace('Montréal','Montreal').replace('Québec','Quebec')
normalized_visits = normalized_visits[~normalized_visits['city'].isin(['Orlando','Dallas','Hamilton','Mississauga','Oklahoma City'])]
normalized_visits_dwtn = normalized_visits[normalized_visits["metric"]=="downtown"].copy()
normalized_visits_dwtn = normalized_visits_dwtn[normalized_visits_dwtn["normalized_visits_by_total_visits"].notna()]
normalized_visits_dwtn['cluster'] = normalized_visits_dwtn['city'].map(dict(zip(clusters['city'], clusters['cluster']))).astype(int)
normalized_visits_dwtn = normalized_visits_dwtn.sort_values(by='week')
normalized_visits_dwtn['rolling'] = normalized_visits_dwtn.groupby('city')['normalized_visits_by_total_visits'].transform(lambda x : x.rolling(window=15).mean().shift(-7))
#normalized_visits_dwtn['cluster'] = normalized_visits_dwtn['cluster'].map({0:0,1:1,2:2,3:3,4:4,5:0})
normalized_visits_dwtn['cluster'] = normalized_visits_dwtn['cluster'].map({0:"R1.1",1:"R1.5",2:"R1.3",3:"R1.4",4:"R1.2",5:"R1.1"})
normalized_visits_dwtn = normalized_visits_dwtn[normalized_visits_dwtn['week']>'2020-06-01']
normalized_visits_dwtn = normalized_visits_dwtn[normalized_visits_dwtn['week']<'2021-06-01']
normalized_visits_dwtn["week"] = pd.to_datetime(normalized_visits_dwtn["week"])
normalized_visits_dwtn_avg = normalized_visits_dwtn.groupby(['week','cluster']).mean()['rolling'].reset_index()
sns.set(rc={'figure.figsize':(7.5,12)})
sns.set(font_scale = 1.15)
sns.set_style("whitegrid")
sns.lineplot(data=normalized_visits_dwtn, x ='week', y = 'rolling', hue='cluster', palette='tab10', units='city', alpha=0.25, estimator=None, linewidth=1.5)#, ci='sd')
sns.lineplot(data=normalized_visits_dwtn_avg, x ='week', y = 'rolling', hue='cluster', palette='tab10', units='cluster', alpha=0.8, estimator=None, linewidth=2.5)#, ci='sd')
plt.ylabel('Location Quotient')
plt.xlabel('Date (Year-Month)')
plt.ylim(0.1,1.6)
plt.legend(bbox_to_anchor=(1.02, 1), loc='upper left', borderaxespad=0)
plt.show()

In [ ]:
display(get_table_as_pandas_df('rq_dwtn_clusters_1015_period_2'))

In [ ]:
sns.set_theme(style="whitegrid")
clusters = get_table_as_pandas_df('rq_dwtn_clusters_1015_period_2').replace('Montréal','Montreal').replace('Québec','Quebec')
normalized_visits = get_table_as_pandas_df('metrics_1015').replace('Montréal','Montreal').replace('Québec','Quebec')
normalized_visits = normalized_visits[~normalized_visits['city'].isin(['Orlando','Dallas','Hamilton','Mississauga','Oklahoma City'])]
normalized_visits_dwtn = normalized_visits[normalized_visits["metric"]=="downtown"].copy()
normalized_visits_dwtn = normalized_visits_dwtn[normalized_visits_dwtn["normalized_visits_by_total_visits"].notna()]
normalized_visits_dwtn['cluster'] = normalized_visits_dwtn['city'].map(dict(zip(clusters['city'], clusters['cluster']))).astype(int)
normalized_visits_dwtn["week"] = pd.to_datetime(normalized_visits_dwtn["week"])
normalized_visits_dwtn = normalized_visits_dwtn.sort_values(by='week')
normalized_visits_dwtn['rolling'] = normalized_visits_dwtn.groupby('city')['normalized_visits_by_total_visits'].transform(lambda x : x.rolling(window=15).mean().shift(-7))
normalized_visits_dwtn['cluster'] = normalized_visits_dwtn['cluster'].map({0:0,1:1,2:2,3:3,4:4,5:3})
normalized_visits_dwtn = normalized_visits_dwtn[normalized_visits_dwtn['week']>'2021-06-01']
normalized_visits_dwtn = normalized_visits_dwtn[normalized_visits_dwtn['week']<'2022-05-15']
normalized_visits_dwtn["week"] = pd.to_datetime(normalized_visits_dwtn["week"])
normalized_visits_dwtn_avg = normalized_visits_dwtn.groupby(['week','cluster']).mean()['rolling'].reset_index()
sns.set(rc={'figure.figsize':(7.5,12)})
sns.set(font_scale = 1.15)
sns.set_style("whitegrid")
sns.lineplot(data=normalized_visits_dwtn, x ='week', y = 'rolling', hue='cluster', palette='husl', units='city', alpha=0.25, estimator=None, linewidth=1.5)#, ci='sd')
sns.lineplot(data=normalized_visits_dwtn_avg, x ='week', y = 'rolling', hue='cluster', palette='husl', units='cluster', alpha=0.8, estimator=None, linewidth=2.5)#, ci='sd')

plt.ylabel('Location Quotient')
plt.xlabel('Date (Year-Month)')
plt.ylim(0.1,1.6)
plt.legend(bbox_to_anchor=(1.02, 1), loc='upper left', borderaxespad=0)
plt.show()

In [ ]:
sns.set_theme(style="whitegrid")
clusters = get_table_as_pandas_df('rq_dwtn_clusters_0823_period_2').replace('Montréal','Montreal').replace('Québec','Quebec')
normalized_visits = get_table_as_pandas_df('0714_combined_metrics_df').replace('Montréal','Montreal').replace('Québec','Quebec')
normalized_visits = normalized_visits[~normalized_visits['city'].isin(['Orlando','Dallas','Hamilton','Mississauga','Oklahoma City'])]
normalized_visits_dwtn = normalized_visits[normalized_visits["metric"]=="downtown"].copy()
normalized_visits_dwtn = normalized_visits_dwtn[~normalized_visits_dwtn["normalized_visits_by_total_visits"].isna()]
normalized_visits_dwtn = normalized_visits_dwtn[normalized_visits_dwtn['week']>'2021-06-01']
normalized_visits_dwtn = normalized_visits_dwtn[normalized_visits_dwtn['week']<'2022-05-31']
normalized_visits_dwtn["week"] = pd.to_datetime(normalized_visits_dwtn["week"])
normalized_visits_dwtn = normalized_visits_dwtn.sort_values(by='week')
normalized_visits_dwtn['cluster'] = normalized_visits_dwtn['city'].map(dict(zip(clusters['city'], clusters['cluster']))).astype(int)
normalized_visits_dwtn['normalized_visits_by_total_visits'] = normalized_visits_dwtn['normalized_visits_by_total_visits'].astype(float)
normalized_visits_dwtn['rolling'] = normalized_visits_dwtn.groupby('city')['normalized_visits_by_total_visits'].transform(lambda x : x.rolling(window=15).mean().shift(-7))
normalized_visits_dwtn['cluster'] = normalized_visits_dwtn['cluster'].map({0:0,1:1,2:2,3:3,4:4,5:1})
normalized_visits_dwtn_avg = normalized_visits_dwtn.groupby(['week','cluster']).mean()['rolling'].reset_index()
sns.set(rc={'figure.figsize':(7.5,12)})
sns.set(font_scale = 1.15)
sns.set_style("whitegrid")
sns.lineplot(data=normalized_visits_dwtn, x ='week', y = 'rolling', hue='cluster', palette='husl', units='city', alpha=0.25, estimator=None, linewidth=1.5)#, ci='sd')
sns.lineplot(data=normalized_visits_dwtn_avg, x ='week', y = 'rolling', hue='cluster', palette='husl', units='cluster', alpha=0.8, estimator=None, linewidth=2.5)#, ci='sd')
plt.ylabel('Location Quotient')
plt.xlabel('Date (Year-Month)')
plt.ylim(0.1,1.6)
plt.legend(bbox_to_anchor=(1.02, 1), loc='upper left', borderaxespad=0)
plt.show()

In [ ]:
import seaborn as sns
sns.set_theme(style="whitegrid")
clusters = get_table_as_pandas_df('rq_dwtn_clusters_0831_all_2').replace('Montréal','Montreal').replace('Québec','Quebec')
normalized_visits = get_table_as_pandas_df('0714_combined_metrics_df').replace('Montréal','Montreal').replace('Québec','Quebec')
normalized_visits = normalized_visits[~normalized_visits['city'].isin(['Orlando','Dallas','Hamilton','Mississauga','Oklahoma City'])]
normalized_visits_dwtn = normalized_visits[normalized_visits["metric"]=="downtown"].copy()
normalized_visits_dwtn = normalized_visits_dwtn[normalized_visits_dwtn["normalized_visits_by_total_visits"].notna()]
normalized_visits_dwtn['cluster'] = normalized_visits_dwtn['city'].map(dict(zip(clusters['city'], clusters['cluster']))).astype(int)
normalized_visits_dwtn = normalized_visits_dwtn.sort_values(by='week')
normalized_visits_dwtn['rolling'] = normalized_visits_dwtn.groupby('city')['normalized_visits_by_total_visits'].transform(lambda x : x.rolling(window=15).mean().shift(-7))
normalized_visits_dwtn['cluster'] = normalized_visits_dwtn['cluster'].map({0:0,1:1,2:2,3:3,4:4,5:0})
normalized_visits_dwtn = normalized_visits_dwtn[normalized_visits_dwtn['week']>'2020-06-01']
normalized_visits_dwtn = normalized_visits_dwtn[normalized_visits_dwtn['week']<'2021-06-01']
normalized_visits_dwtn["week"] = pd.to_datetime(normalized_visits_dwtn["week"])
normalized_visits_dwtn_avg = normalized_visits_dwtn.groupby(['week','cluster']).mean()['rolling'].reset_index()
sns.set(rc={'figure.figsize':(15,12)})
sns.set(font_scale = 1.15)
sns.set_style("whitegrid")
sns.lineplot(data=normalized_visits_dwtn, x ='week', y = 'rolling', hue='cluster', palette='tab10', units='city', alpha=0.25, estimator=None, linewidth=1.5)#, ci='sd')
sns.lineplot(data=normalized_visits_dwtn_avg, x ='week', y = 'rolling', hue='cluster', palette='tab10', units='cluster', alpha=0.8, estimator=None, linewidth=2.5)#, ci='sd')
plt.ylabel('Location Quotient')
plt.xlabel('Date (Year-Month)')
plt.ylim(0.1,1.6)
plt.legend(bbox_to_anchor=(1.02, 1), loc='upper left', borderaxespad=0)
plt.show()

In [ ]:
display(normalized_visits_dwtn)

In [ ]:
normalized_visits_dwtn.groupby(['week','cluster']).mean()['rolling'].reset_index()

In [ ]:
clusters = get_table_as_pandas_df('rq_dwtn_clusters_0823_period_1').replace('Montréal','Montreal').replace('Québec','Quebec')
normalized_visits_dwtn = normalized_visits[normalized_visits["metric"]=="downtown"].copy()
normalized_visits_dwtn['cluster'] = normalized_visits_dwtn['city'].map(dict(zip(clusters['city'], clusters['cluster']))).astype(int)
mean_dwtn = normalized_visits_dwtn.groupby(['cluster','week']).mean()['normalized_visits_by_total_visits'].reset_index()
mean_dwtn["week"] = pd.to_datetime(mean_dwtn["week"]).dt.date
mean_dwtn = mean_dwtn[mean_dwtn["week"]>datetime.date(2020, 6, 1)]
mean_dwtn['rolling'] = mean_dwtn.groupby('cluster')['normalized_visits_by_total_visits'].transform(lambda x : x.rolling(window=9).mean().shift(-4))
sns.set(rc={'figure.figsize':(14,10)})
ax = sns.lineplot(data=mean_dwtn, x ='week', y = 'rolling', hue='cluster', palette='tab10')
plt.ylabel('Downtown RQ')
plt.xlabel('Date')
plt.show()

In [ ]:
clusters = get_table_as_pandas_df('rq_dwtn_clusters_0823_period_2').replace('Montréal','Montreal').replace('Québec','Quebec')
normalized_visits_dwtn = normalized_visits[normalized_visits["metric"]=="downtown"].copy()
normalized_visits_dwtn['cluster'] = normalized_visits_dwtn['city'].map(dict(zip(clusters['city'], clusters['cluster']))).astype(int)
mean_dwtn = normalized_visits_dwtn.groupby(['cluster','week']).mean()['normalized_visits_by_total_visits'].reset_index()
mean_dwtn["week"] = pd.to_datetime(mean_dwtn["week"]).dt.date
mean_dwtn = mean_dwtn[mean_dwtn["week"]>datetime.date(2020, 6, 1)]
mean_dwtn['rolling'] = mean_dwtn.groupby('cluster')['normalized_visits_by_total_visits'].transform(lambda x : x.rolling(window=9).mean().shift(-4))
sns.set(rc={'figure.figsize':(14,10)})
ax = sns.lineplot(data=mean_dwtn, x ='week', y = 'rolling', hue='cluster', palette='tab10')
plt.ylabel('Downtown RQ')
plt.xlabel('Date')
plt.show()